# Build model V0.0

# Net Data
    
| batch=1 | subdivisions=1 | Training |
| :---: | :---: | :---: |
| decay=0.0005 | angle=0 | saturation = 1.5 |
| batch=64 | subdivisions=16 | width=608 |
| height=608 | channels=3 | momentum=0.9 |
|exposure = 1.5 | hue=.1 | learning_rate=0.001|
|burn_in=1000| max_batches = 500200| policy=steps|  
|steps=400000,450000|scales=.1,.1||
 

In [1]:
import cv2
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense, Conv2D, LeakyReLU, ZeroPadding2D
from keras.layers import concatenate
from keras.activations import linear
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adagrad
from keras.utils import plot_model
from keras.engine.topology import Layer
import tensorflow as tf
from Yolo_layer import YoloLayer

Using TensorFlow backend.


# Global Variables

In [2]:
labels = ['person'       , 'bicycle'      , 'car'          , 'motorcycle',
          'airplane'     , 'bus'          , 'train'        , 'truck', 
          'boat'         , 'traffic light', 'fire hydrant' ,'stop sign',
          'parking meter', 'bench'        , 'bird'         , 'cat', 
          'dog'          , 'horse'        , 'sheep'        ,'cow' , 
          'elephant'     , 'bear'         , 'zebra'        ,'giraffe',
          'backpack'     , 'umbrella'     , 'handbag'      ,'tie',
          'suitcase'     , 'frisbee'      , 'skis'         ,'snowboard',
          'sports ball'  , 'kite'         , 'baseball bat' , 'baseball glove',
          'skateboard'   , 'surfboard'    , 'tennis racket','bottle',
          'wine glass'   , 'cup'          , 'fork'         ,'knife',
          'spoon'        , 'bowl'         , 'banana'       ,'apple',
          'sandwich'     , 'orange'       , 'broccoli'     ,'carrot',
          'hot dog'      , 'pizza'        , 'donut'        ,'cake',
          'chair'        , 'couch'        , 'potted plant' ,'bed',
          'dining table' , 'toilet'       , 'tv'           ,'laptop', 
          'mouse'        , 'remote'       , 'keyboard'     ,'cell phone',
          'microwave'    , 'oven'         , 'toaster'      ,'sink', 
          'refrigerator' , 'book'         , 'clock'        , 'vase',
          'scissors'     , 'teddy bear'   , 'hair drier'   , 'toothbrush']

IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13 , 13
BOX              = 5
nb_class         = len(labels)

anchors          = [10,13,16,30,33,23,30,61,62,45,59,119,116,90,156,198,373,326]

noobj_scale      = 1.0
obj_scale        = 5.0
COORD_SCALE      = 1.0
class_scale      = 1.0

batch_size       = 16
warmup_batches   = 0
TRUE_BOX_BUFFER  = 50


ALPHA = 0.1
max_grid = 480
max_box_per_image = 20

# Inputs

In [3]:
# net_h, net_w, 3
input_image = Input(shape=(None, None, 3)) 
true_boxes  = Input(shape=(1, 1, 1, max_box_per_image, 4))
# grid_h, grid_w, nb_anchor, pc+bx+by+bh+bw+nb_class
true_yolo_1 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 
true_yolo_2 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 
true_yolo_3 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 

# Prediction Layer

In [4]:
# Conv
# Layer 1

lay1_1 = Conv2D(32, 3, strides=(1, 1), padding='same')(input_image)
lay1_2 = BatchNormalization(epsilon=0.001)(lay1_1)
lay1_3 = LeakyReLU(alpha=ALPHA)(lay1_2)

# Downsample

# Conv
# Layer 2

lay2_1 = ZeroPadding2D(((1,0),(1,0)))(lay1_3)
lay2_2 = Conv2D(64, 3, strides=(2, 2), padding='valid')(lay2_1)
lay2_3 = BatchNormalization(epsilon=0.001)(lay2_2)
lay2_4 = LeakyReLU(alpha=ALPHA)(lay2_3)

# Conv
# Layer 3

lay3_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay2_4)
lay3_2 = BatchNormalization(epsilon=0.001)(lay3_1)
lay3_3 = LeakyReLU(alpha=ALPHA)(lay3_2)

# Conv
# Layer 4

lay4_1 = Conv2D(64, 3, strides=(1, 1), padding='same')(lay3_3)
lay4_2 = BatchNormalization(epsilon=0.001)(lay4_1)
lay4_3 = LeakyReLU(alpha=ALPHA)(lay4_2)

# Shortcut -> Conv
# Layer 5

lay5_0 = concatenate([lay2_4,lay4_3], axis=-1)
lay5_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay5_0)
lay5_2 = linear(lay5_1)

# Downsample

# Conv
# Layer 6

lay6_1 = ZeroPadding2D(((1,0),(1,0)))(lay5_2)
lay6_2 = Conv2D(128, 3, strides=(2, 2), padding='valid')(lay6_1)
lay6_3 = BatchNormalization(epsilon=0.001)(lay6_2)
lay6_4 = LeakyReLU(alpha=ALPHA)(lay6_3)

# Conv
# Layer 7

lay7_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay6_4)
lay7_2 = BatchNormalization(epsilon=0.001)(lay7_1)
lay7_3 = LeakyReLU(alpha=ALPHA)(lay7_2)

# Conv
# Layer 8

lay8_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay7_3)
lay8_2 = BatchNormalization(epsilon=0.001)(lay8_1)
lay8_3 = LeakyReLU(alpha=ALPHA)(lay8_2)

# Shortcut -> Conv
# Layer 9
lay9_0 = concatenate([lay6_3,lay8_3], axis=-1)
lay9_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay9_0)
lay9_2 = linear(lay9_1)

# Conv
# Layer 10

lay10_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay9_2)
lay10_2 = BatchNormalization(epsilon=0.001)(lay10_1)
lay10_3 = LeakyReLU(alpha=ALPHA)(lay10_2)

# Conv
# Layer 11

lay11_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay10_3)
lay11_2 = BatchNormalization(epsilon=0.001)(lay11_1)
lay11_3 = LeakyReLU(alpha=ALPHA)(lay11_2)

# Shortcut -> Conv
# layer 12

lay12_0 = concatenate([lay9_2,lay11_3], axis=-1)
lay12_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay12_0)
lay12_2 = linear(lay9_1)

# Downsample

# Conv 
# Layer 13

lay13_1 = ZeroPadding2D(((1,0),(1,0)))(lay12_2)
lay13_2 = Conv2D(256, 3, strides=(2, 2), padding='valid')(lay13_1)
lay13_3 = BatchNormalization(epsilon=0.001)(lay13_2)
lay13_4 = LeakyReLU(alpha=ALPHA)(lay13_3)

# Conv
# Layer 14

lay14_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay13_4)
lay14_2 = BatchNormalization(epsilon=0.001)(lay14_1)
lay14_3 = LeakyReLU(alpha=ALPHA)(lay14_2)

# Conv
# Layer 15

lay15_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay14_3)
lay15_2 = BatchNormalization(epsilon=0.001)(lay15_1)
lay15_3 = LeakyReLU(alpha=ALPHA)(lay15_2)


# Shortcut -> Conv
# Layer 16

lay16_0 = concatenate([lay13_4,lay15_3], axis=-1)
lay16_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay16_0)
lay16_2 = linear(lay16_1)


# Conv
# Layer 17

lay17_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay16_2)
lay17_2 = BatchNormalization(epsilon=0.001)(lay17_1)
lay17_3 = LeakyReLU(alpha=ALPHA)(lay17_2)

# Conv
# Layer 18

lay18_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay17_3)
lay18_2 = BatchNormalization(epsilon=0.001)(lay18_1)
lay18_3 = LeakyReLU(alpha=ALPHA)(lay18_2)

# Shortcut -> Conv
# Layer 19

lay19_0 = concatenate([lay16_2,lay18_3], axis=-1)
lay19_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay19_0)
lay19_2 = linear(lay19_1)

# Conv
# Layer 20

lay20_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay19_2)
lay20_2 = BatchNormalization(epsilon=0.001)(lay20_1)
lay20_3 = LeakyReLU(alpha=ALPHA)(lay20_2)

# Conv
# Layer 21

lay21_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay20_3)
lay21_2 = BatchNormalization(epsilon=0.001)(lay21_1)
lay21_3 = LeakyReLU(alpha=ALPHA)(lay21_2)

# Shortcut -> Conv
# Layer 22

lay22_0 = concatenate([lay19_2,lay21_3], axis=-1)
lay22_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay22_0)
lay22_2 = linear(lay22_1)

# Conv
# Layer 23

lay23_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay22_2)
lay23_2 = BatchNormalization(epsilon=0.001)(lay23_1)
lay23_3 = LeakyReLU(alpha=ALPHA)(lay23_2)

# Conv 
# Layer 24 

lay24_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay23_3)
lay24_2 = BatchNormalization(epsilon=0.001)(lay24_1)
lay24_3 = LeakyReLU(alpha=ALPHA)(lay24_2)


# Shortcut -> Conv
# Layer 25

lay25_0 = concatenate([lay22_2,lay24_3], axis=-1)
lay25_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay25_0)
lay25_2 = linear(lay25_1)


# Conv
# Layer 26

lay26_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay25_2)
lay26_2 = BatchNormalization(epsilon=0.001)(lay26_1)
lay26_3 = LeakyReLU(alpha=ALPHA)(lay26_2)


# Conv
# Layer 27

lay27_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay26_2)
lay27_2 = BatchNormalization(epsilon=0.001)(lay27_1)
lay27_3 = LeakyReLU(alpha=ALPHA)(lay27_2)

# Shortcut -> Conv
# Layer 28

lay28_0 = concatenate([lay27_3,lay25_2], axis=-1)
lay28_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay28_0)
lay28_2 = linear(lay28_1)

# Conv
# Layer 29

lay29_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay28_2)
lay29_2 = BatchNormalization(epsilon=0.001)(lay29_1)
lay29_3 = LeakyReLU(alpha=ALPHA)(lay29_2)

# Conv
# Layer 30

lay30_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay29_3)
lay30_2 = BatchNormalization(epsilon=0.001)(lay30_1)
lay30_3 = LeakyReLU(alpha=ALPHA)(lay30_2)

# # Shortcut -> Conv
# Layer 31

lay31_0 = concatenate([lay30_3,lay28_2], axis=-1)
lay31_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay31_0)
lay31_2 = linear(lay31_1)


# Conv
# Layer 32

lay32_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay31_2)
lay32_2 = BatchNormalization(epsilon=0.001)(lay32_1)
lay32_3 = LeakyReLU(alpha=ALPHA)(lay32_2)


# Conv
# Layer 33

lay33_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay32_3)
lay33_2 = BatchNormalization(epsilon=0.001)(lay33_1)
lay33_3 = LeakyReLU(alpha=ALPHA)(lay33_2)

# # Shortcut -> Conv
# Layer 34

lay34_0 = concatenate([lay33_3,lay31_2], axis=-1)
lay34_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay34_0)
lay34_2 = linear(lay34_1)

# Conv
# Layer 35

lay35_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay34_2)
lay35_2 = BatchNormalization(epsilon=0.001)(lay33_1)
lay35_3 = LeakyReLU(alpha=ALPHA)(lay33_2)

# Conv
# Layer 36

lay36_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay35_3)
lay36_2 = BatchNormalization(epsilon=0.001)(lay36_1)
lay36_3 = LeakyReLU(alpha=ALPHA)(lay36_2)

# # Shortcut -> Conv
# Layer 37

lay37_0 = concatenate([lay36_3,lay34_2], axis=-1)
lay37_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay37_0)
lay37_2 = linear(lay37_1)

# Downsample

# Conv
# Layer 38

lay38_1 = ZeroPadding2D(((1,0),(1,0)))(lay37_2)
lay38_2 = Conv2D(512, 3, strides=(2, 2), padding='valid')(lay38_1)
lay38_3 = BatchNormalization(epsilon=0.001)(lay38_2)
lay38_4 = LeakyReLU(alpha=ALPHA)(lay38_3)

# Conv
# Layer 39

lay39_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay38_4)
lay39_2 = BatchNormalization(epsilon=0.001)(lay39_1)
lay39_3 = LeakyReLU(alpha=ALPHA)(lay39_2)

# Conv
# Layer 40

lay40_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay39_3)
lay40_2 = BatchNormalization(epsilon=0.001)(lay40_1)
lay40_3 = LeakyReLU(alpha=ALPHA)(lay40_2)


# Shortcut -> Conv
# Layer 41

lay41_0 = concatenate([lay40_3,lay38_4], axis=-1)
lay41_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay41_0)
lay41_2 = linear(lay41_1)

# Conv
# Layer 42

lay42_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay41_2)
lay42_2 = BatchNormalization(epsilon=0.001)(lay42_1)
lay42_3 = LeakyReLU(alpha=ALPHA)(lay42_2)


# Conv
# Layer 43

lay43_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay42_2)
lay43_2 = BatchNormalization(epsilon=0.001)(lay43_1)
lay43_3 = LeakyReLU(alpha=ALPHA)(lay43_2)

# # Shortcut -> Conv
# Layer 44

lay44_1 = concatenate([lay43_3,lay41_2], axis=-1)
lay44_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay44_1)
lay44_3 = linear(lay44_2)


# Conv
# Layer 45

lay45_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay44_3)
lay45_2 = BatchNormalization(epsilon=0.001)(lay45_1)
lay45_3 = LeakyReLU(alpha=ALPHA)(lay45_2)

# Conv
# Layer 46

lay46_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay45_3)
lay46_2 = BatchNormalization(epsilon=0.001)(lay46_1)
lay46_3 = LeakyReLU(alpha=ALPHA)(lay46_2)

# # Shortcut -> Conv
# Layer 47

lay47_1 = concatenate([lay46_3,lay44_3], axis=-1)
lay47_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay47_1)
lay47_3 = linear(lay47_2)


# Conv
# Layer 48

lay48_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay47_3)
lay48_2 = BatchNormalization(epsilon=0.001)(lay48_1)
lay48_3 = LeakyReLU(alpha=ALPHA)(lay48_2)

# Conv
# Layer 49

lay49_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay48_3)
lay49_2 = BatchNormalization(epsilon=0.001)(lay49_1)
lay49_3 = LeakyReLU(alpha=ALPHA)(lay49_2)

# # Shortcut -> Conv
# Layer 50

lay50_1 = concatenate([lay49_3,lay47_3], axis=-1)
lay50_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay50_1)
lay50_3 = linear(lay50_2)

# Conv
# Layer 51

lay51_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay50_3)
lay51_2 = BatchNormalization(epsilon=0.001)(lay51_1)
lay51_3 = LeakyReLU(alpha=ALPHA)(lay51_2)

# Conv
# Layer 52

lay52_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay51_3)
lay52_2 = BatchNormalization(epsilon=0.001)(lay52_1)
lay52_3 = LeakyReLU(alpha=ALPHA)(lay52_2)

# # Shortcut -> Conv
# Layer 53

lay53_1 = concatenate([lay52_3,lay50_2], axis=-1)
lay53_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay53_1)
lay53_3 = linear(lay53_2)


# Conv
# Layer 54

lay54_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay53_3)
lay54_2 = BatchNormalization(epsilon=0.001)(lay54_1)
lay54_3 = LeakyReLU(alpha=ALPHA)(lay54_2)

# Conv
# Layer 55

lay55_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay54_3)
lay55_2 = BatchNormalization(epsilon=0.001)(lay55_1)
lay55_3 = LeakyReLU(alpha=ALPHA)(lay55_2)

# # Shortcut -> Conv
# Layer 56

lay56_1 = concatenate([lay55_3,lay53_3], axis=-1)
lay56_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay56_1)
lay56_3 = linear(lay56_2)


# Conv
# Layer 57

lay57_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay56_3)
lay57_2 = BatchNormalization(epsilon=0.001)(lay57_1)
lay57_3 = LeakyReLU(alpha=ALPHA)(lay57_2)

# Conv
# Layer 58

lay58_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay57_3)
lay58_2 = BatchNormalization(epsilon=0.001)(lay58_1)
lay58_3 = LeakyReLU(alpha=ALPHA)(lay58_2)

# # Shortcut -> Conv
# Layer 59

lay59_1 = concatenate([lay58_3,lay56_3], axis=-1)
lay59_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay59_1)
lay59_3 = linear(lay59_2)


# Conv
# Layer 60

lay60_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay59_3)
lay60_2 = BatchNormalization(epsilon=0.001)(lay60_1)
lay60_3 = LeakyReLU(alpha=ALPHA)(lay60_2)

# Conv
# Layer 61

lay61_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay60_3)
lay61_2 = BatchNormalization(epsilon=0.001)(lay61_1)
lay61_3 = LeakyReLU(alpha=ALPHA)(lay61_2)

# # Shortcut -> Conv
# Layer 62

lay62_1 = concatenate([lay61_3,lay59_3], axis=-1)
lay62_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay62_1)
lay62_3 = linear(lay62_2)

# # Downsample

# Conv
# Layer 63

lay63_1 = ZeroPadding2D(((1,0),(1,0)))(lay62_3)
lay63_2 = Conv2D(1024, 3, strides=(2, 2), padding='valid')(lay63_1)
lay63_3 = BatchNormalization(epsilon=0.001)(lay63_2)
lay63_4 = LeakyReLU(alpha=ALPHA)(lay63_3)

# Conv
# Layer 64

lay64_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay63_4)
lay64_2 = BatchNormalization(epsilon=0.001)(lay64_1)
lay64_3 = LeakyReLU(alpha=ALPHA)(lay64_2)

# Conv
# Layer 65

lay65_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay64_3)
lay65_2 = BatchNormalization(epsilon=0.001)(lay65_1)
lay65_3 = LeakyReLU(alpha=ALPHA)(lay65_2)

# Shortcut -> Conv
# Layer 66

lay66_1 = concatenate([lay65_3,lay63_3], axis=-1)
lay66_2 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay66_1)
lay66_3 = linear(lay66_2)

# Conv
# Layer 67

lay67_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay66_3)
lay67_2 = BatchNormalization(epsilon=0.001)(lay67_1)
lay67_3 = LeakyReLU(alpha=ALPHA)(lay67_2)

# Conv
# Layer 68

lay68_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay67_3)
lay68_2 = BatchNormalization(epsilon=0.001)(lay65_1)
lay68_3 = LeakyReLU(alpha=ALPHA)(lay65_2)

# # Shortcut -> Conv
# Layer 69

lay69_1 = concatenate([lay68_3,lay66_3], axis=-1)
lay69_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay69_1)
lay69_3 = linear(lay69_2)

# Conv
# Layer 70

lay70_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay69_3)
lay70_2 = BatchNormalization(epsilon=0.001)(lay70_1)
lay70_3 = LeakyReLU(alpha=ALPHA)(lay70_2)

# Conv
# Layer 71

lay71_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay70_3)
lay71_2 = BatchNormalization(epsilon=0.001)(lay71_1)
lay71_3 = LeakyReLU(alpha=ALPHA)(lay71_2)

# # Shortcut -> Conv
# Layer 72

lay72_1 = concatenate([lay71_3,lay69_3], axis=-1)
lay72_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay72_1)
lay72_3 = linear(lay72_2)


# Conv
# Layer 73

lay73_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay72_3)
lay73_2 = BatchNormalization(epsilon=0.001)(lay73_1)
lay73_3 = LeakyReLU(alpha=ALPHA)(lay73_2)


# Conv
# Layer 74

lay74_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay73_3)
lay74_2 = BatchNormalization(epsilon=0.001)(lay74_1)
lay74_3 = LeakyReLU(alpha=ALPHA)(lay74_2)

# Shortcut -> Conv
# Layer 75

lay75_1 = concatenate([lay74_3,lay72_3], axis=-1)
lay75_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay75_1)
lay75_3 = linear(lay75_2)

# ######################

# Conv
# Layer 76

lay76_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay75_3)
lay76_2 = BatchNormalization(epsilon=0.001)(lay76_1)
lay76_3 = LeakyReLU(alpha=ALPHA)(lay76_2)

# Conv
# Layer 77

lay77_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay76_3)
lay77_2 = BatchNormalization(epsilon=0.001)(lay77_1)
lay77_3 = LeakyReLU(alpha=ALPHA)(lay77_2)

# Conv
# Layer 78

lay78_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay77_3)
lay78_2 = BatchNormalization(epsilon=0.001)(lay78_1)
lay78_3 = LeakyReLU(alpha=ALPHA)(lay78_2)

# Conv
# Layer 79

lay79_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay78_3)
lay79_2 = BatchNormalization(epsilon=0.001)(lay79_1)
lay79_3 = LeakyReLU(alpha=ALPHA)(lay79_2)

# Conv
# Layer 80

lay80_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay79_3)
lay80_2 = BatchNormalization(epsilon=0.001)(lay80_1)
lay80_3 = LeakyReLU(alpha=ALPHA)(lay80_2)

# Conv
# Layer 81

lay81_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay80_3)
lay81_2 = BatchNormalization(epsilon=0.001)(lay81_1)
lay81_3 = LeakyReLU(alpha=ALPHA)(lay81_2)

# Conv
# Layer 82

lay82_1 = Conv2D(255, 1, strides=(1, 1), padding='same')(lay81_3)
lay82_2 = BatchNormalization(epsilon=0.001)(lay82_1)
lay82_3 = LeakyReLU(alpha=ALPHA)(lay82_2)

pred_yolo_1 = Conv2D((3*(5+nb_class)), 1, strides=(1, 1), padding='same')(lay82_3)
ignore_thresh = .7

loss_yolo_1 = YoloLayer(anchors[12:], 
                        [1*num for num in max_grid], 
                        batch_size, 
                        warmup_batches, 
                        ignore_thresh, 
                        grid_scales[0],
                        obj_scale,
                        noobj_scale,
                        xywh_scale,
                        class_scale)([input_image, pred_yolo_1, 
                                      true_yolo_1, true_boxes])
# [yolo] 83
# mask = 6,7,8

# classes=80
# num=9
# jitter=.3

# truth_thresh = 1
# random=1


# [route] 84
# layers = -4

# Conv
# Layer 85

# lay85_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay84_3)
# lay85_2 = BatchNormalization(epsilon=0.001)(lay85_1)
# lay85_3 = LeakyReLU(alpha=ALPHA)(lay85_2)

# [upsample] 86
# stride=2

# [route] 87
# layers = -1, 61



# Conv
# Layer 88

# lay88_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay87_3)
# lay88_2 = BatchNormalization(epsilon=0.001)(lay88_1)
# lay88_3 = LeakyReLU(alpha=ALPHA)(lay88_2)

# Conv
# Layer 89

# lay89_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay88_3)
# lay89_2 = BatchNormalization(epsilon=0.001)(lay89_1)
# lay89_3 = LeakyReLU(alpha=ALPHA)(lay89_2)

# Conv
# Layer 90

# lay90_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay89_3)
# lay90_2 = BatchNormalization(epsilon=0.001)(lay90_1)
# lay90_3 = LeakyReLU(alpha=ALPHA)(lay90_2)

# Conv
# Layer 91

# lay90_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay89_3)
# lay90_2 = BatchNormalization(epsilon=0.001)(lay90_1)
# lay90_3 = LeakyReLU(alpha=ALPHA)(lay90_2)

# Conv
# Layer 92

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 93

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=512
# activation=leaky

# Conv
# Layer 94

# size=1
# stride=1
# pad=1
# filters=255
# activation=linear


# [yolo] 95
# mask = 3,4,5
# anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
# classes=80
# num=9
# jitter=.3
# ignore_thresh = .7
# truth_thresh = 1
# random=1



# [route] 96
# layers = -4

# Conv
# Layer 97

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# [upsample] 98
# stride=2

# [route] 99
# layers = -1, 36



# Conv 100
# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 101

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# Conv
# Layer 102

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 103

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# Conv
# Layer 104

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 105

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# Conv
# Layer 106

# size=1
# stride=1
# pad=1
# filters=255
# activation=linear


# [yolo] 107
# mask = 0,1,2
# anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
# classes=80
# num=9
# jitter=.3
# ignore_thresh = .7
# truth_thresh = 1
# random=1



TypeError: 'int' object is not iterable

In [ ]:
model = Model(inputs=input_image, outputs=lay82_3)
plot_model(model, to_file='model.png',show_shapes='true')